# 1) Install & Import

In [1]:
# General
# --------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.graph_objects as go
from time import time

# Time Series
# --------------------------------
!pip install utilsforecast window_ops sktime

# window ops is time series statistics alternative to pandas/numpy with numba jitted wrapper for faster speed
from window_ops.rolling import rolling_mean, rolling_std, seasonal_rolling_mean, seasonal_rolling_std
from window_ops.ewm import ewm_mean

# utilsforecast is Nixtla forecast library to support various forecasting activity
from utilsforecast.plotting import plot_series
from utilsforecast.losses import mae, rmse, mape
from utilsforecast.evaluation import evaluate
from utilsforecast.preprocessing import fill_gaps

# for encoding seasonal day of month dummy
from sklearn.preprocessing import OneHotEncoder



# 2) Data Load & Cleaning

## a. Load Data

In [2]:
#Load the train  & test data

train = pd.read_excel("Train_Masked.xlsx")
train['ds'] = pd.to_datetime(train['ds'])

test = pd.read_excel("Test_Masked.xlsx")
test['ds'] = pd.to_datetime(test['ds'])

print(train.shape)
print(train.tail(2))
print(test.shape)
print(test.tail(2))

(7561, 3)
     unique_id         ds             y
7559         w 2023-11-01  1.371468e+08
7560         x 2023-11-01  5.859256e+09
(168, 3)
    unique_id         ds             y
166         g 2023-11-21  4.849416e+08
167         g 2023-11-20  7.802561e+08


In [ ]:
#check type

print(train.dtypes)
print(test.dtypes)

unique_id            object
ds           datetime64[ns]
y                     int64
dtype: object
unique_id            object
ds           datetime64[ns]
y                     int64
dtype: object


Sort Data by each unique_id & ds

In [3]:
# Sort data by each unique_id & ds

train = train.sort_values(['unique_id', 'ds']).reset_index(drop=True)
print(train.head(2))

test = test.sort_values(['unique_id', 'ds']).reset_index(drop=True)
print(test.head(2))

  unique_id         ds             y
0         a 2023-02-15  1.648533e+07
1         a 2023-02-16  1.373972e+07
  unique_id         ds             y
0         a 2023-11-20  2.374394e+07
1         a 2023-11-21  2.482359e+07


## b. Fill Missing Data

### i. Fill gabs each unique_id using global start & end

In [4]:
# both train & test don't have missing data

print(train.isna().sum().sum())
print(test.isna().sum().sum())

0
0


In [5]:
# instead each unique_id has different start and end ds
# test data already has the same start and end ds

train.groupby('unique_id')['y'].nunique()

unique_id
a    278
b    285
c    323
d    323
e    323
f    323
g    303
h    322
i    323
j    323
k    323
l    322
m    322
n    319
o    257
p    323
q    323
r    318
s    318
t    323
u    310
v    317
w    323
x    323
Name: y, dtype: int64

In [6]:
# make all unique_id has the same start and end ds
# same end ds only is also possible

train_ds_adjusted = fill_gaps(
                      train,
                      freq='D',
                      start='global', # set per_serie as alternative
                      end='global'
)

In [7]:
# now train data has missing value

train_ds_adjusted.isna().sum()

unique_id      0
ds             0
y            191
dtype: int64

### ii. Fill missing data with linear interpolation & backward fill

**`Important note:`**


---


Outlier removal won't be applied in this experiment. Since many suppliers have meaningfull outlier such as:
1. k, g, q, j, & x every 20th
2. c every end of month 29th-31st
3. p, r, t every payday 25th
4. etc...

In [8]:
# pivot each unique_id to column since imputing data is done for each unique_id

pivot_train = train_ds_adjusted.pivot(index='ds', columns='unique_id', values='y').reset_index()
pivot_train.head()

unique_id,ds,a,b,c,d,e,f,g,h,i,...,o,p,q,r,s,t,u,v,w,x
0,2023-01-01,NaN,NaN,5.231384e+07,2.044831e+08,4.404235e+07,3.191533e+06,7.718303e+06,4.870798e+07,1.748822e+06,...,NaN,9.651130e+07,7.411590e+06,5.443212e+08,NaN,2.992602e+07,63861.666667,755433.333333,1.258351e+08,3.451310e+09
1,2023-01-02,NaN,NaN,5.116290e+07,1.981253e+08,5.324734e+07,3.851228e+06,1.007228e+08,1.073309e+08,1.724640e+06,...,NaN,2.525272e+08,2.210810e+07,5.906870e+08,NaN,3.634868e+07,172400.000000,313466.666667,1.397711e+08,3.731446e+09
2,2023-01-03,NaN,NaN,5.334606e+07,1.857693e+08,1.327295e+07,3.637375e+06,3.443108e+07,1.220318e+08,2.656570e+06,...,NaN,1.945267e+08,2.070964e+07,4.988815e+08,NaN,3.719193e+07,88686.666667,424633.333333,1.258863e+08,3.519401e+09
3,2023-01-04,NaN,NaN,7.601435e+07,1.334978e+08,1.646031e+07,3.900159e+06,6.075249e+06,1.398422e+08,1.732746e+06,...,NaN,1.485548e+08,1.673417e+07,5.071721e+08,NaN,2.567895e+07,53703.333333,80333.333333,1.251337e+08,3.580602e+09
4,2023-01-05,NaN,NaN,1.080719e+08,1.424359e+08,2.160851e+07,3.334608e+06,1.175494e+07,2.404745e+08,1.515165e+06,...,NaN,1.510916e+08,1.762962e+07,5.303884e+08,NaN,2.519460e+07,79166.666667,243133.333333,1.269201e+08,3.674095e+09


In [9]:
# show which unqiue_id has missing value

pivot_train.isna().sum()

unique_id
ds     0
a     45
b     38
c      0
d      0
e      0
f      0
g     20
h      1
i      0
j      0
k      0
l      1
m      0
n      4
o     65
p      0
q      0
r      5
s      5
t      0
u      7
v      0
w      0
x      0
dtype: int64

In [10]:
# first, linearly interpolate is applied for every missing value from start to end ds
# second, all missing value before start ds is filled by backward fill

for unique_id in pivot_train.drop("ds",axis=1).columns:
    pivot_train[unique_id] = pivot_train[unique_id].interpolate(method="linear")
    pivot_train[unique_id] = pivot_train[unique_id].fillna(method="bfill")

In [11]:
# no more missing value

pivot_train.isna().sum().sum()

0

In [12]:
# convert back into long data

unpivoted_train = pd.melt(pivot_train, id_vars=['ds'], var_name='unique_id', value_name='y')
unpivoted_train.head()

,ds,unique_id,y
0,2023-01-01,a,1.648533e+07
1,2023-01-02,a,1.648533e+07
2,2023-01-03,a,1.648533e+07
3,2023-01-04,a,1.648533e+07
4,2023-01-05,a,1.648533e+07


In [13]:
# compare the shape between orginal and preprocessed train

print(train.shape)
print(unpivoted_train.shape)

(7561, 3)
(7752, 3)


In [14]:
# copy preprocessed data to train

train = unpivoted_train.copy()
train = train[['unique_id','ds','y']]
train.head()

,unique_id,ds,y
0,a,2023-01-01,1.648533e+07
1,a,2023-01-02,1.648533e+07
2,a,2023-01-03,1.648533e+07
3,a,2023-01-04,1.648533e+07
4,a,2023-01-05,1.648533e+07


### iii. Add seasonal dummy

In [15]:
# extract day of month
trainX = train.copy()
trainX['day_of_month'] = trainX['ds'].dt.day
print(trainX.head(2))

testX = test.copy()
testX['day_of_month'] = testX['ds'].dt.day
print(testX.head(2))

  unique_id         ds             y  day_of_month
0         a 2023-01-01  1.648533e+07             1
1         a 2023-01-02  1.648533e+07             2
  unique_id         ds             y  day_of_month
0         a 2023-11-20  2.374394e+07            20
1         a 2023-11-21  2.482359e+07            21


In [16]:
# trainX------------------------------------------------------

categories = [trainX['day_of_month'].unique()]

encoder = OneHotEncoder(sparse_output=False,
                            drop="first",
                            categories=categories,
                            )

encoded_cols = encoder.fit_transform(trainX[['day_of_month']])
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(['day_of_month']))

# Concatenate the encoded columns with the original data
trainX = pd.concat([trainX[['unique_id', 'ds','y']],encoded_df], axis=1)

print(trainX.shape)
trainX.head()

(7752, 33)


,unique_id,ds,y,day_of_month_2,day_of_month_3,day_of_month_4,day_of_month_5,day_of_month_6,day_of_month_7,day_of_month_8,...,day_of_month_22,day_of_month_23,day_of_month_24,day_of_month_25,day_of_month_26,day_of_month_27,day_of_month_28,day_of_month_29,day_of_month_30,day_of_month_31
0,a,2023-01-01,1.648533e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,a,2023-01-02,1.648533e+07,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,a,2023-01-03,1.648533e+07,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a,2023-01-04,1.648533e+07,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a,2023-01-05,1.648533e+07,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# testX------------------------------------------------------

encoded_cols = encoder.transform(testX[['day_of_month']])
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(['day_of_month']))

# Concatenate the encoded columns with the original data
testX = pd.concat([testX[['unique_id', 'ds','y']],encoded_df], axis=1)

print(testX.shape)
testX.head()

(168, 33)


,unique_id,ds,y,day_of_month_2,day_of_month_3,day_of_month_4,day_of_month_5,day_of_month_6,day_of_month_7,day_of_month_8,...,day_of_month_22,day_of_month_23,day_of_month_24,day_of_month_25,day_of_month_26,day_of_month_27,day_of_month_28,day_of_month_29,day_of_month_30,day_of_month_31
0,a,2023-11-20,2.374394e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,a,2023-11-21,2.482359e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,a,2023-11-22,2.737511e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a,2023-11-23,2.111852e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a,2023-11-24,2.471033e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## c. Plot Data

In [19]:
fig = plot_series(
    train, # DataFrame with columns [id_col, time_col, target_col]
    #test,
    ids=None,
    max_insample_length=None, # Maximum number of train/insample observations to be plotted.
    engine='plotly', # Library used to plot. ‘plotly’, ‘plotly-resampler’ or ‘matplotlib’.
    palette='viridis', # Name of the matplotlib colormap to use
    )

fig.update_layout(
    #width=1600,
    height=900,
)

fig.show(renderer="colab")

# 3)  Statistical Approach

In [20]:
# Statistical Approach
# --------------------------------
!pip install statsforecast

from statsforecast import StatsForecast
from statsforecast.models import (
    # Previous Models----------------------
    #AutoRegressive,
    #WindowAverage,
    # Simple Models----------------------
    #HistoricAverage,
    #Naive,
    #SeasonalNaive,
    # Classicall Models----------------------
    AutoARIMA,
    AutoETS,
    #AutoCES,
    #AutoTheta,
    # Multiple Seasonality Model----------------------
    #MSTL,
    # Sparse / Inttermitent Model----------------------
    CrostonOptimized
)

# for building confidence interval
from statsforecast.utils import ConformalIntervals

C:\Users\alisu\anaconda3\Lib\site-packages\statsforecast\core.py:25: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)



## a. Define Model Object

In [26]:
# similar to cross validation, but instead we use the result to build confidence interval
intervals = ConformalIntervals(h=7, n_windows=5)

sf_models = [
    # Previous Models------------------
    #AutoRegressive(lags=[1]),
    #WindowAverage(window_size=7),
    # Simple Models--------------------
    #HistoricAverage(),
    #Naive(),
    #SeasonalNaive(season_length=30),
    # Classicall Models---------------
    AutoARIMA(season_length=7,prediction_intervals=intervals),
    #AutoETS(season_length=7,prediction_intervals=intervals),
    #AutoCES(season_length=7,prediction_intervals=intervals),
    #AutoTheta(season_length=7,prediction_intervals=intervals),
    # Multiple Seasonality Model------
    #MSTL(
        #season_length=[7, 30],
        #trend_forecaster=AutoARIMA()),
    # Sparse / Inttermitent Model-----
    #CrostonOptimized(prediction_intervals=intervals),
]

## b. Define Forecast Object

In [27]:
sf = StatsForecast(
    df=trainX, #trainX
    models=sf_models, # model used to fit each time series
    freq='D', # frequency of the data
    n_jobs = -1
)

## c. Forecast

In [28]:
# Create future exogenous regressors

X_df = testX.drop(['y'],axis=1)
print(X_df.shape)
X_df.head()

(168, 32)


,unique_id,ds,day_of_month_2,day_of_month_3,day_of_month_4,day_of_month_5,day_of_month_6,day_of_month_7,day_of_month_8,day_of_month_9,...,day_of_month_22,day_of_month_23,day_of_month_24,day_of_month_25,day_of_month_26,day_of_month_27,day_of_month_28,day_of_month_29,day_of_month_30,day_of_month_31
0,a,2023-11-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,a,2023-11-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,a,2023-11-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a,2023-11-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a,2023-11-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
init = time()

horizon=7 # how many days to forecast
level= [80] # how much % of confidence interval

sf_forecast = sf.forecast(
    h=horizon,
    X_df=X_df,
    level=level
)

end = time()
time_run = (end - init)

print(f'Time: {time_run:.2f} seconds')
sf_forecast = sf_forecast.reset_index()
sf_forecast.head()

Time: 399.07 seconds


,unique_id,ds,AutoARIMA,AutoARIMA-lo-80,AutoARIMA-hi-80
0,a,2023-11-20,24347440.0,22042354.0,26652526.0
1,a,2023-11-21,18252042.0,13985666.0,22518418.0
2,a,2023-11-22,17599076.0,9842128.0,25356022.0
3,a,2023-11-23,21666244.0,16821506.0,26510984.0
4,a,2023-11-24,21194858.0,15805044.0,26584672.0


In [30]:
# Combine forecast output with actual data

sf_forecast = pd.concat([sf_forecast,test[['y']]], axis=1)
sf_forecast.head()

,unique_id,ds,AutoARIMA,AutoARIMA-lo-80,AutoARIMA-hi-80,y
0,a,2023-11-20,24347440.0,22042354.0,26652526.0,2.374394e+07
1,a,2023-11-21,18252042.0,13985666.0,22518418.0,2.482359e+07
2,a,2023-11-22,17599076.0,9842128.0,25356022.0,2.737511e+07
3,a,2023-11-23,21666244.0,16821506.0,26510984.0,2.111852e+07
4,a,2023-11-24,21194858.0,15805044.0,26584672.0,2.471033e+07


In [ ]:
# Export forecast output

#sf_forecast.to_excel('sf_forecast.xlsx')

In [31]:
fig = plot_series(
    train, # DataFrame with columns [id_col, time_col, target_col]
    sf_forecast,
    ids=None, # Time Series to plot. If None, time series are selected randomly.
    max_ids=8, # Maximum number of ids to plot.
    max_insample_length=7*28, # Maximum number of train/insample observations to be plotted.
    engine='plotly', # Library used to plot. ‘plotly’, ‘plotly-resampler’ or ‘matplotlib’.
    palette='viridis', # Name of the matplotlib colormap to use
    models=None,
    level=[80]
    )

fig.update_layout(
    #width=1600,
    height=900,
)

fig.show(renderer="colab")

##  d. Evaluate

In [32]:
metric = [
    #mae,
    rmse,
    #mape
]

In [34]:
eval_sf = evaluate(
    sf_forecast,
    metrics=metric,
)
eval_sf.head()

,unique_id,metric,AutoARIMA
0,a,rmse,5.190184e+06
1,b,rmse,1.285807e+07
2,c,rmse,6.352523e+07
3,d,rmse,8.066515e+07
4,e,rmse,1.940487e+07


In [ ]:
# export forecast evaluation

#eval_sf.to_excel('eval_sf.xlsx')

## e. Cross Validation

In [ ]:
init = time()

sf_cv = sf.cross_validation(
    df = trainX,
    h = 7,
    step_size = 7,
    n_windows = 5,
  )

end = time()
time_run = (end - init)

print(f'Time: {time_run:.2f} seconds')
sf_cv.head()

In [ ]:
sf_cv = sf_cv.reset_index()
sf_cv.head()

## f. CV Evaluate

In [ ]:
metric = [
    #mae,
    rmse,
    #mape
]

In [ ]:
def evaluate_cross_validation(df, metric):
    models = sf_cv.drop(columns=['unique_id', 'ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    # Calculate loss for every unique_id and cutoff.
    for cutoff in sf_cv['cutoff'].unique():
        eval_ = evaluate(sf_cv[sf_cv['cutoff'] == cutoff], metrics=[metric], models=models)
        evals.append(eval_)
    evals = pd.concat(evals)
    evals = evals.groupby('unique_id').mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals


In [ ]:
eval_sf_cv = evaluate_cross_validation(sf_cv.reset_index(), rmse)

eval_sf_cv

# 4)  FP Prophet Approach

In [ ]:
!pip install prophet

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

## a. Data Formating

In [ ]:
train_fb = train.copy()
train_fb.tail()

In [ ]:
pivot_train_fb = train_fb.pivot(index='ds', columns='unique_id', values='y').reset_index()
pivot_train_fb.tail()

In [ ]:
pivot_train_fb.tail(28)

## b. Forecast

In [ ]:
init = time()

fb_forecast = pd.DataFrame() #for storing the result for every iteration (Supplier/Product)

for unique_id in pivot_train_fb.drop("ds",axis=1).columns:
    try:
        train_fb_model = pivot_train_fb[['ds',unique_id]] # Taking the data
        train_fb_model.columns = ['ds','y'] # Reformat the data as required by FB Prophet Model
        train_fb_model['ds'] = pd.to_datetime(train_fb_model['ds']) # Change text date to datetime object

        forecaster = Prophet(
            growth="linear", #linear(d) or logistic
            scaling="minmax",
            changepoints = None, #None (d) if want to add manual specify the date
            n_changepoints = 25,
            changepoint_range=0.8, #0.8 (d)
            holidays= None, #or holidays
            seasonality_mode='additive', #or multipicative
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10.00,
            holidays_prior_scale=10.00,
            yearly_seasonality= "auto", #or False
            weekly_seasonality= "auto", #or False
            daily_seasonality= "auto", #or False
            )#.add_seasonality(
                #name="monthly",
                #period=30.5,
                #fourier_order=4
            #).add_seasonality(
                #name="daily",
                #period=1,
                #fourier_order=15
            #).add_seasonality(
                #name="weekly",
                #period=7,
                #fourier_order=20
            #).add_seasonality(
                #name="yearly",
                #period=365.25,
                #fourier_order=20
            #).add_seasonality(
                #name="quarterly",
                #period=365.25/4,
                #fourier_order=20,
                #prior_scale =15
            #)

        forecaster.fit(train_fb_model) # Fitting the model
        future = forecaster.make_future_dataframe(periods=7, freq='D',include_history=False) # Creating forecast dataframe # freq can be D, H, W, Y etc
        forecast = forecaster.predict(future) # predict future value

        fb_forecast[unique_id] = forecast[["yhat"]].tail(7) # Gather the result (alternative: yhat_lower, yhat, and yhat_upper)
    except:
        pass

end = time()
time_run = (end - init)

print(f'Time: {time_run:.2f} seconds')

In [ ]:
fb_forecast['ds'] = future.tail(7)
fb_forecast.head()

In [ ]:
fb_forecast = pd.melt(fb_forecast, id_vars=['ds'], var_name='unique_id', value_name='y')
fb_forecast.head()

In [ ]:
fb =  fb_forecast.copy()
fb = fb.rename(columns={'y':'prophet (7)'})
fb.head()

## c. Cross Validation

In [ ]:
cutoffs = pd.to_datetime(['2023-10-23'])

fb_cv = cross_validation(
    forecaster,
    horizon='7 days',
    cutoffs=cutoffs
)

## d. Evaluate

# 5)  Machine Learning Approach

In [35]:
# Machine Learning Approach
# --------------------------------
!pip install mlforecast lightgbm xgboost

from mlforecast import MLForecast
from mlforecast.utils import PredictionIntervals
from mlforecast.target_transforms import Differences, LocalBoxCox, LocalMinMaxScaler

from mlforecast.lgb_cv import LightGBMCV

import lightgbm as lgb
#import xgboost as xgb

# Hyperparameter Tuning
!pip install optuna

import optuna

## a. Feature Engineering

In [36]:
feature_config = dict(
    freq="D", # daily frequency data
    target_transforms=[Differences([1])], # minus all value with previous day to detrending (tree model cannot extrapolate trend)
    lags=[0 + (i+1) for i in range(31)], # lag values from previous day to 31 days
    lag_transforms={
        1: [(ewm_mean, 0.3)],
        7: [(ewm_mean, 0.3),(rolling_mean, 7),(rolling_std, 7),(rolling_mean, 14),(rolling_std, 14),(seasonal_rolling_mean,7, 3),(seasonal_rolling_std,7, 3)],
        14:[(ewm_mean, 0.3),(rolling_mean, 7),(rolling_std, 7),(rolling_mean, 14),(rolling_std, 14),(seasonal_rolling_mean,7, 2),(seasonal_rolling_std,7, 2)],
        21:[(ewm_mean, 0.3),(rolling_mean, 7),(rolling_std, 7)],
        29:[(ewm_mean, 0.3)],
        30:[(ewm_mean, 0.3)],
        31:[(ewm_mean, 0.3)],
    },
    date_features=['month', 'day', 'dayofweek', 'week','quarter','dayofyear'], #extract date feature
)

mlf = MLForecast(
          models=[], # model will constructed next
          **feature_config,
          num_threads=4,
)

In [37]:
prep = mlf.preprocess(train)
print(train.shape)
print(prep.shape)
prep.tail()

(7752, 3)
(6984, 61)


,unique_id,ds,y,lag1,lag2,lag3,lag4,lag5,lag6,lag7,...,rolling_std_lag21_window_size7,ewm_mean_lag29_alpha0.3,ewm_mean_lag30_alpha0.3,ewm_mean_lag31_alpha0.3,month,day,dayofweek,week,quarter,dayofyear
7747,x,2023-11-15,3.535860e+08,1.908825e+07,2.460735e+08,-1.973553e+08,-1.994507e+08,3.104101e+08,5.730876e+07,-1.361076e+08,...,1.640193e+09,-3.370868e+07,1.634589e+08,-1.535327e+07,11,15,2,46,4,319
7748,x,2023-11-16,-6.763809e+08,3.535860e+08,1.908825e+07,2.460735e+08,-1.973553e+08,-1.994507e+08,3.104101e+08,5.730876e+07,...,1.637830e+09,1.521581e+08,-3.370868e+07,1.634589e+08,11,16,3,46,4,320
7749,x,2023-11-17,-1.210107e+08,-6.763809e+08,3.535860e+08,1.908825e+07,2.460735e+08,-1.973553e+08,-1.994507e+08,3.104101e+08,...,1.156066e+09,1.505744e+08,1.521581e+08,-3.370868e+07,11,17,4,46,4,321
7750,x,2023-11-18,3.416212e+08,-1.210107e+08,-6.763809e+08,3.535860e+08,1.908825e+07,2.460735e+08,-1.973553e+08,-1.994507e+08,...,3.542594e+08,8.453124e+08,1.505744e+08,1.521581e+08,11,18,5,46,4,322
7751,x,2023-11-19,9.288805e+08,3.416212e+08,-1.210107e+08,-6.763809e+08,3.535860e+08,1.908825e+07,2.460735e+08,-1.973553e+08,...,3.071986e+08,-3.305767e+08,8.453124e+08,1.505744e+08,11,19,6,46,4,323


## b. Cross Validation with LGBMVC & Hyperparameter Tuning with Optuna

In [38]:
fixed_params = {
    'objective': 'regression',
    'boosting': "gbdt",
    'bagging_freq': 1,
    'verbose': -1,
    'linear_tree': False,
    'linear_lambda':0,
    'force_col_wise': True, # because number of columns is large
    'bagging_freq' : 1,
    'metric':'',
    'n_estimators': 1000,
    'learning_rate': 0.1, #
    'max_depth':-1 #
}

In [39]:
def objective(trial):
    cv = LightGBMCV(**feature_config, num_threads=12)
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 2500),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
        #'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        #'max_depth': trial.suggest_int('max_depth', 5, 20),
    }
    hist = cv.setup(
        train,
        params={**fixed_params, **params},
        n_windows=5,
        h=7,
        metric='rmse'
    )
    hist = []
    early_stopping_evals = 2
    early_stopping_pct = 0.01
    eval_every = 20
    for i in range(0, fixed_params['n_estimators'], eval_every):
        val = cv.partial_fit(eval_every)
        trial.report(val, step=i)
        rounds = eval_every + i
        hist.append((rounds, val))
        if trial.should_prune():
            raise optuna.TrialPruned()
        #if cv.should_stop(hist, early_stopping_evals, early_stopping_pct):
            #print(f"Early stopping at round {rounds:,}")
            #best_iter = cv._best_iter(hist, early_stopping_evals)
            #trial.set_user_attr('n_estimators', best_iter)
            break
    return val

In [40]:
init = time()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30, n_jobs=1)

end = time()
time_run = (end - init)

print(f'Time: {time_run:.2f} seconds')

[I 2023-12-23 18:16:06,641] A new study created in memory with name: no-name-25be4c92-ae29-4bc9-b201-b6b6bba0aa2d
[I 2023-12-23 18:16:49,097] Trial 0 finished with value: 80313362.56217991 and parameters: {'num_leaves': 1439, 'lambda_l1': 0.019278434682866448, 'lambda_l2': 0.0018697723474761, 'bagging_fraction': 0.7163577515278541, 'feature_fraction': 0.7490702918310586, 'min_data_in_leaf': 75}. Best is trial 0 with value: 80313362.56217991.
[I 2023-12-23 18:17:42,258] Trial 1 finished with value: 82706514.56218801 and parameters: {'num_leaves': 1265, 'lambda_l1': 7.192128540811753, 'lambda_l2': 4.588164942004746e-05, 'bagging_fraction': 0.8545212440556665, 'feature_fraction': 0.942963580124861, 'min_data_in_leaf': 97}. Best is trial 0 with value: 80313362.56217991.
[I 2023-12-23 18:19:43,693] Trial 2 finished with value: 80454715.72998178 and parameters: {'num_leaves': 2116, 'lambda_l1': 1.402671077126005e-05, 'lambda_l2': 0.41577019681393745, 'bagging_fraction': 0.8652197681488225, '

Time: 879.22 seconds


In [41]:
study.best_params

{'num_leaves': 2089,
 'lambda_l1': 0.0881168631523731,
 'lambda_l2': 0.0017629174978789357,
 'bagging_fraction': 0.7490233860912122,
 'feature_fraction': 0.6931598053620143,
 'min_data_in_leaf': 15}

In [ ]:
study.best_value

331510784.6281371

In [42]:
study.trials_dataframe().head()

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_feature_fraction,params_lambda_l1,params_lambda_l2,params_min_data_in_leaf,params_num_leaves,state
0,0,8.031336e+07,2023-12-23 18:16:06.642588,2023-12-23 18:16:49.097177,0 days 00:00:42.454589,0.716358,0.749070,1.927843e-02,1.869772e-03,75,1439,COMPLETE
1,1,8.270651e+07,2023-12-23 18:16:49.098344,2023-12-23 18:17:42.258225,0 days 00:00:53.159881,0.854521,0.942964,7.192129e+00,4.588165e-05,97,1265,COMPLETE
2,2,8.045472e+07,2023-12-23 18:17:42.259226,2023-12-23 18:19:43.692861,0 days 00:02:01.433635,0.865220,0.947527,1.402671e-05,4.157702e-01,30,2116,COMPLETE
3,3,7.824760e+07,2023-12-23 18:19:43.693860,2023-12-23 18:20:07.796997,0 days 00:00:24.103137,0.529695,0.746448,5.918989e-06,3.634617e-08,99,268,COMPLETE
4,4,8.348865e+07,2023-12-23 18:20:07.797999,2023-12-23 18:20:53.904453,0 days 00:00:46.106454,0.898037,0.775906,1.217661e-08,6.252762e-05,88,1785,COMPLETE


## c. Analyzing Tuned Parameted

In [43]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show(renderer="colab")

In [44]:
fig = optuna.visualization.plot_param_importances(study)
fig.show(renderer="colab")

In [46]:
fig = optuna.visualization.plot_rank(study)

fig.update_layout(
    #width=1600,
    height=900,
)

fig.show(renderer="colab")

C:\Users\alisu\AppData\Local\Temp\ipykernel_15924\3674502780.py:1: ExperimentalWarning:

plot_rank is experimental (supported from v3.2.0). The interface can change in the future.



## d. Forecast

In [47]:
best_params = {
    **fixed_params,
    **study.best_params,
}

fcst = MLForecast(
    models=lgb.LGBMRegressor(**best_params),
    **feature_config,
    num_threads=4,
)

fcst.fit(train,
        prediction_intervals=PredictionIntervals(n_windows=5, h=7)
)

MLForecast(models=[LGBMRegressor], freq=<Day>, lag_features=['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6', 'lag7', 'lag8', 'lag9', 'lag10', 'lag11', 'lag12', 'lag13', 'lag14', 'lag15', 'lag16', 'lag17', 'lag18', 'lag19', 'lag20', 'lag21', 'lag22', 'lag23', 'lag24', 'lag25', 'lag26', 'lag27', 'lag28', 'lag29', 'lag30', 'lag31', 'ewm_mean_lag1_alpha0.3', 'ewm_mean_lag7_alpha0.3', 'rolling_mean_lag7_window_size7', 'rolling_std_lag7_window_size7', 'rolling_mean_lag7_window_size14', 'rolling_std_lag7_window_size14', 'seasonal_rolling_mean_lag7_season_length7_window_size3', 'seasonal_rolling_std_lag7_season_length7_window_size3', 'ewm_mean_lag14_alpha0.3', 'rolling_mean_lag14_window_size7', 'rolling_std_lag14_window_size7', 'rolling_mean_lag14_window_size14', 'rolling_std_lag14_window_size14', 'seasonal_rolling_mean_lag14_season_length7_window_size2', 'seasonal_rolling_std_lag14_season_length7_window_size2', 'ewm_mean_lag21_alpha0.3', 'rolling_mean_lag21_window_size7', 'rolling_std_lag21_w

In [51]:
levels = [80]
ml_forecasts = fcst.predict(7, level=levels)
ml_forecasts.tail()

,unique_id,ds,LGBMRegressor,LGBMRegressor-lo-80,LGBMRegressor-hi-80
163,x,2023-11-22,3.838287e+09,3.363647e+09,4.312927e+09
164,x,2023-11-23,3.670741e+09,3.008633e+09,4.332848e+09
165,x,2023-11-24,3.294662e+09,1.995273e+09,4.594050e+09
166,x,2023-11-25,3.473726e+09,3.148231e+09,3.799221e+09
167,x,2023-11-26,2.513221e+09,1.708499e+09,3.317944e+09


In [52]:
# Combine forecast output with actual data

ml_forecasts = pd.concat([ml_forecasts,test[['y']]], axis=1)
ml_forecasts.head()

,unique_id,ds,LGBMRegressor,LGBMRegressor-lo-80,LGBMRegressor-hi-80,y
0,a,2023-11-20,2.340589e+07,1.725827e+07,2.955351e+07,2.374394e+07
1,a,2023-11-21,1.805123e+07,7.986117e+06,2.811634e+07,2.482359e+07
2,a,2023-11-22,1.632496e+07,5.376162e+06,2.727376e+07,2.737511e+07
3,a,2023-11-23,2.249192e+07,1.388700e+07,3.109684e+07,2.111852e+07
4,a,2023-11-24,2.633305e+07,1.371580e+07,3.895031e+07,2.471033e+07


In [53]:
# Combine statistical & machine learning forecast

forecasts = pd.concat([sf_forecast,ml_forecasts[['LGBMRegressor','LGBMRegressor-lo-80','LGBMRegressor-hi-80']]], axis=1)
forecasts.tail()

,unique_id,ds,AutoARIMA,AutoARIMA-lo-80,AutoARIMA-hi-80,y,LGBMRegressor,LGBMRegressor-lo-80,LGBMRegressor-hi-80
163,x,2023-11-22,3.842694e+09,3.259388e+09,4.426001e+09,2.709496e+09,3.838287e+09,3.363647e+09,4.312927e+09
164,x,2023-11-23,3.865922e+09,3.038922e+09,4.692921e+09,2.170458e+09,3.670741e+09,3.008633e+09,4.332848e+09
165,x,2023-11-24,3.869206e+09,2.654325e+09,5.084087e+09,1.967317e+09,3.294662e+09,1.995273e+09,4.594050e+09
166,x,2023-11-25,4.094876e+09,3.763824e+09,4.425928e+09,1.829916e+09,3.473726e+09,3.148231e+09,3.799221e+09
167,x,2023-11-26,4.091787e+09,3.388969e+09,4.794606e+09,1.856352e+09,2.513221e+09,1.708499e+09,3.317944e+09


In [65]:
fig = plot_series(
    train, # DataFrame with columns [id_col, time_col, target_col]
    forecasts,
    ids=None, # Time Series to plot. If None, time series are selected randomly.
    max_ids=8, # Maximum number of ids to plot.
    max_insample_length=7*8, # Maximum number of train/insample observations to be plotted.
    engine='plotly', # Library used to plot. ‘plotly’, ‘plotly-resampler’ or ‘matplotlib’.
    palette='viridis', # Name of the matplotlib colormap to use
    level=[80],
    )

fig.update_layout(
    #width=1600,
    height=800,
)

fig.show(renderer="colab")

In [55]:
forecasts.drop("ds",axis=1).groupby('unique_id').mean().transpose()[['a']]

unique_id,a
AutoARIMA,2.308173e+07
AutoARIMA-lo-80,1.830871e+07
AutoARIMA-hi-80,2.785474e+07
y,2.454313e+07
LGBMRegressor,2.134053e+07
LGBMRegressor-lo-80,1.207997e+07
LGBMRegressor-hi-80,3.060109e+07


## d. Evaluate

In [56]:
metric = [
    #mae,
    rmse,
    #mape
]

In [59]:
eval_ml = evaluate(
    forecasts,
    metrics=metric,
)
print(eval_ml[['AutoARIMA','LGBMRegressor']].sum())
eval_ml

AutoARIMA        4.052777e+09
LGBMRegressor    2.474605e+09
dtype: float64


,unique_id,metric,AutoARIMA,LGBMRegressor
0,a,rmse,5.190184e+06,5.618922e+06
1,b,rmse,1.285807e+07,8.629575e+05
2,c,rmse,6.352523e+07,6.759212e+07
3,d,rmse,8.066515e+07,4.571190e+07
4,e,rmse,1.940487e+07,1.088282e+07
5,f,rmse,2.174351e+06,2.843759e+06
6,g,rmse,3.579682e+08,4.220216e+08
7,h,rmse,1.283906e+07,2.324341e+07
8,i,rmse,5.293781e+06,3.680445e+06
9,j,rmse,9.993010e+08,2.300145e+07


In [ ]:
eval_ml.to_excel('eval_ml.xlsx')

## e. Cross Validation

In [ ]:
best_params = {
    **fixed_params,
    **study.best_params,
}

cv = LightGBMCV(
    **feature_config,
    num_threads=4,
)

cv_hist = cv.fit(
    train,
    n_windows=5,
    h=7,
    params={**best_params},
    early_stopping_evals = 2,
    early_stopping_pct = 0.01,
    eval_every = 20,
    compute_cv_preds=True,
    metric='rmse'
)

In [ ]:
cv.cv_preds_

## f. CV Evaluate

In [ ]:
metric = [
    #mae,
    rmse,
    #mape
]

In [ ]:
def evaluate_cross_validation(df, metric):
    models = cv.cv_preds_.drop(columns=['unique_id', 'ds', 'window', 'y']).columns.tolist()
    evals = []
    # Calculate loss for every unique_id and cutoff.
    for window in cv.cv_preds_['window'].unique():
        eval_ = evaluate(cv.cv_preds_[cv.cv_preds_['window'] == window], metrics=[metric], models=models)
        evals.append(eval_)
    evals = pd.concat(evals)
    evals = evals.groupby('unique_id').mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals


In [ ]:
lgbmcv_cv = evaluate_cross_validation(cv.cv_preds_.reset_index(), mape)

lgbmcv_cv